<a href="https://colab.research.google.com/github/portkata/KataGo/blob/master/JBX2010_template_60bKatago_bot_1po_OGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please click "COPY to Drive" on top and save as your own copy first. Please also download the required Config files from Github. https://github.com/JBX2010/KataGo_Colab/

1. Please first check what GPU is allocated by Colab by clicking the command line below.

In [ ]:
!nvidia-smi

2. Building KataGo and gtp2ogs. Please switch CUDA/OPENCL depending on the GPU allocated. Search KATAGO_BACKEND = "OPENCL" and update accordingly. For T4 recommend to use CUDA. For P100, P4 and K80 recommend to use OPENCL.  

In [ ]:
KATAGO_BACKEND="CUDA"
%cd /content
!apt install sudo
!sudo apt remove cmake
!sudo apt purge --auto-remove cmake
!mkdir ~/temp
%cd ~/temp
!wget https://cmake.org/files/v3.12/cmake-3.12.3-Linux-x86_64.sh
!sudo mkdir /opt/cmake
!sudo sh cmake-3.12.3-Linux-x86_64.sh --prefix=/opt/cmake --skip-license
!sudo rm -R /usr/local/bin/cmake
!sudo ln -s /opt/cmake/bin/cmake /usr/local/bin/cmake
%cd /content
!apt-get update
!add-apt-repository -y ppa:ubuntu-toolchain-r/test
!apt-get -y install gcc-8 g++-8 zlib1g-dev libzip-dev libboost-filesystem-dev libgoogle-perftools-dev
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 40 --slave /usr/bin/g++ g++ /usr/bin/g++-8 
!apt-get install curl
!curl -sL https://deb.nodesource.com/setup_13.x |  -E bash -
!apt-get install -y nodejs
!npm install -g gtp2ogs
%cd /content
!mkdir testtt 
%cd /content/testtt
!git clone -b devel https://github.com/online-go/gtp2ogs 
%cd /content/testtt/gtp2ogs 
!git branch 
!sudo cp -rf * /usr/lib/node_modules/gtp2ogs/
%cd /usr/lib/node_modules/gtp2ogs/ 
!npm install
%cd /content
!git clone https://github.com/lightvector/KataGo.git 
!cd /content/KataGo/cpp/ && cmake . -DBUILD_MCTS=1 -DUSE_BACKEND=$KATAGO_BACKEND && make 
!cd /content/KataGo/cpp/ && wget https://github.com/portkata/KataGo/releases/download/v0.1/60.bin.gz
!cd /content/KataGo/cpp/ && gunzip 60.bin.gz
!cd /content/KataGo/cpp/ && mv 60.bin 60e.bin
!cd /content/KataGo/cpp/ && wget https://github.com/portkata/KataGo/releases/download/v.01/t1p1v1.cfg.gz
!cd /content/KataGo/cpp/ && gunzip t1p1v1.cfg.gz
!cd /content/KataGo/cpp/ && wget http://res.yikeweiqi.com/yklinker/yk-linker.zip
!cd /content/KataGo/cpp/ && unzip yk-linker.zip


In the above commands, if you want to update the 60b net from the default pre-release one in this script to a stronger one, replace the github address in the following line 

```
!cd /content/KataGo/cpp/ && wget https://github.com/portkata/KataGo/releases/download/v0.1/60.bin.gz
```
with the address for the updated net. then in the line below, replace "60.bin.gz" with the name of the updated net. Then in the next line below, replace the 60 in "60.bin" with the updated net name, but leave the "60e.bin" as is. 


3. Run Bot on OGS https://online-go.com/ . Please update your ogsBotName and your ogsBotAPIKey in below command. Finally click command below to start you bot. You will need to logon to https://online-go.com/ with your local pc or smart phone to see how your bot runs... Enjoy!


In [ ]:
!nodejs /usr/lib/node_modules/gtp2ogs/gtp2ogs.js --username <bot_username> --apikey <api_key> --ogspv KataGo  --persist   --maxconnectedgames 4  --noautohandicapranked --maxhandicap 1 --speeds live --greeting "I use 1 playout. The first move takes 10 seconds. Then it should be very quick. Good luck in Go and all you do today"   -- /content/KataGo/cpp/katago gtp -model /content/KataGo/cpp/60e.bin -config /content/KataGo/cpp/t1p1v1.cfg

# 4a. (Optional) copy and paste the function below  in browser console and press enter to prevent disconnect when browser is inactive 

```
setInterval(ClickConnect,60000)
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)
```
4b. (Optional) Upload pre-tuned P100 OPENCL tunning file can save sometime on the KataGo initialization and may help enhance Bot Strength (IF GPU is P100). File name: tune6_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt . Please upload to the same /KataGo/cpp folder and then click below command.


In [ ]:
%cd /root/
!mkdir .katago 
%cd /root/.katago/ 
!mkdir opencltuning
!mv /content/KataGo/cpp/tune6_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt  /root/.katago/opencltuning/tune6_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt

5. (Optional) KataGo GPU BenchMark testing and Configuration fine tunning (This probably doesn't work as written).

In [ ]:
!cd /content/KataGo/cpp/ && ./katago benchmark  -v 3200 -tune -model 60e.bin -config gtp_example.cfg